# Tutorial 04: Lineshape Analysis and Fitting
## Master EPR Spectral Analysis with EPyR Tools

Welcome to the fourth EPyR Tools tutorial! EPR lineshape analysis is fundamental for extracting quantitative information about paramagnetic systems. This notebook explores the comprehensive `lineshapes` module for spectral fitting and analysis.

### 🎯 Learning Objectives

By the end of this tutorial, you will:
- Understand EPR lineshape theory (Gaussian, Lorentzian, Voigtian)
- Apply single and multi-component spectral fitting
- Extract quantitative parameters (g-values, linewidths, intensities)
- Evaluate fitting quality and parameter uncertainties
- Handle complex multi-site EPR spectra
- Create publication-quality fitted spectrum plots

### 📐 EPR Lineshape Theory

EPR lineshapes arise from different broadening mechanisms:
- **Gaussian**: Inhomogeneous broadening (strain, unresolved hyperfine)
- **Lorentzian**: Homogeneous broadening (lifetime, relaxation)
- **Voigtian**: Convolution of Gaussian + Lorentzian (realistic case)
- **Pseudo-Voigt**: Linear combination approximation to Voigtian

Let's explore these with real EPR data!

## 🚀 Setup and Lineshape Module Exploration

First, let's explore the comprehensive lineshape capabilities in EPyR Tools:

In [ ]:
# Essential imports
import epyr
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.optimize import curve_fit

# Configure matplotlib
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
plt.rcParams['lines.linewidth'] = 2

print(f"EPyR Tools Version: {epyr.__version__}")
print(f"Lineshapes module: {hasattr(epyr, 'lineshapes')}")

# Explore lineshape functions
if hasattr(epyr, 'lineshapes'):
    print("\nAvailable lineshape functions:")
    lineshape_funcs = [func for func in dir(epyr.lineshapes) if not func.startswith('_')]
    for func in lineshape_funcs:
        print(f"  - {func}")
    
    # Import key lineshape functions
    from epyr.lineshapes import (
        gaussian, lorentzian, voigtian, pseudo_voigt,
        Lineshape, fit_epr_signal
    )
    
    print("\n✅ Lineshape functions imported successfully!")
else:
    print("❌ Lineshapes module not available")

# Set up data path
data_path = Path('../data')
print(f"\nData directory exists: {data_path.exists()}")

## 📊 Exploring Individual Lineshape Functions

Let's start by understanding the mathematical properties of different EPR lineshapes:

In [ ]:
# Create field axis for lineshape demonstration
field = np.linspace(3300, 3400, 1000)
center = 3350  # G
width = 10     # G

print("📐 EPR Lineshape Function Comparison")
print("=" * 45)

# Generate different lineshapes
shapes = {
    'Gaussian': gaussian(field, center, width),
    'Lorentzian': lorentzian(field, center, width),
    'Voigtian (σ=5, γ=5)': voigtian(field, center, sigma=5, gamma=5),
    'Pseudo-Voigt (η=0.5)': pseudo_voigt(field, center, width, eta=0.5)
}

# Plot comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

colors = ['blue', 'red', 'green', 'orange']

# Linear scale comparison
for i, (name, shape) in enumerate(shapes.items()):
    ax1.plot(field, shape, color=colors[i], linewidth=2.5, label=name)

ax1.set_xlabel('Magnetic Field (G)')
ax1.set_ylabel('Normalized Intensity')
ax1.set_title('EPR Lineshapes - Linear Scale')
ax1.grid(True, alpha=0.3)
ax1.legend()

# Log scale to show wings
for i, (name, shape) in enumerate(shapes.items()):
    ax2.semilogy(field, shape + 1e-6, color=colors[i], linewidth=2.5, label=name)

ax2.set_xlabel('Magnetic Field (G)')
ax2.set_ylabel('Normalized Intensity (log)')
ax2.set_title('EPR Lineshapes - Log Scale (Wings)')
ax2.grid(True, alpha=0.3)
ax2.legend()

plt.tight_layout()
plt.show()

# Analyze lineshape properties
print(f"\n📊 Lineshape Properties Analysis:")
print(f"{'Shape':<20} {'FWHM (G)':<12} {'Peak Height':<15} {'Wing Behavior':<20}")
print(f"{'-'*20} {'-'*12} {'-'*15} {'-'*20}")

for name, shape in shapes.items():
    # Find FWHM
    peak_idx = np.argmax(shape)
    half_max = shape[peak_idx] / 2
    
    # Find half-maximum points
    left_idx = np.where(shape[:peak_idx] <= half_max)[0]
    right_idx = np.where(shape[peak_idx:] <= half_max)[0]
    
    if len(left_idx) > 0 and len(right_idx) > 0:
        fwhm = field[peak_idx + right_idx[0]] - field[left_idx[-1]]
    else:
        fwhm = np.nan
    
    peak_height = shape[peak_idx]
    
    # Wing behavior (ratio at ±2*FWHM from center)
    wing_field = 2 * width
    center_idx = np.argmin(np.abs(field - center))
    wing_left_idx = np.argmin(np.abs(field - (center - wing_field)))
    wing_right_idx = np.argmin(np.abs(field - (center + wing_field)))
    
    wing_ratio = (shape[wing_left_idx] + shape[wing_right_idx]) / (2 * peak_height)
    
    if 'Gaussian' in name:
        wing_desc = "Exponential decay"
    elif 'Lorentzian' in name:
        wing_desc = "Power law (1/x²)"
    else:
        wing_desc = "Mixed behavior"
    
    print(f"{name:<20} {fwhm:<12.2f} {peak_height:<15.6f} {wing_desc:<20}")

print("\n💡 Key Insights:")
print("  - Gaussian: Narrow core, fast decay in wings (inhomogeneous broadening)")
print("  - Lorentzian: Broader wings, slower decay (homogeneous broadening)")
print("  - Voigtian: Convolution combines both effects (most realistic)")
print("  - Pseudo-Voigt: Fast approximation with adjustable mixing (η parameter)")

print("\n✅ Lineshape comparison complete!")

## 🔬 Loading Real EPR Data for Fitting

Now let's load a real EPR spectrum and prepare it for lineshape analysis:

In [ ]:
# Load real EPR spectrum
epr_file = data_path / '130406SB_CaWO4_Er_CW_5K_20.DSC'

if epr_file.exists():
    print(f"Loading EPR spectrum: {epr_file.name}")
    field_data, intensity_data, params_data, _ = epyr.eprload(str(epr_file))
    
    print(f"\nEPR Spectrum Information:")
    print(f"Field range: {field_data.min():.1f} to {field_data.max():.1f} G")
    print(f"Field points: {len(field_data)}")
    print(f"Microwave frequency: {params_data.get('MWFQ', 'Unknown')} GHz")
    print(f"Temperature: 5 K (from filename)")
    print(f"Sample: Er³⁺ in CaWO4 single crystal")

else:
    print("Real EPR file not found. Creating synthetic EPR spectrum...")
    
    # Create synthetic multi-component EPR spectrum
    field_data = np.linspace(3300, 3400, 800)
    
    # Multi-site Er³⁺ spectrum with different g-values
    component1 = gaussian(field_data, 3340, 8) * 0.6    # Site 1
    component2 = gaussian(field_data, 3355, 12) * 0.8   # Site 2  
    component3 = gaussian(field_data, 3375, 6) * 0.4    # Site 3
    
    # Add some Lorentzian broadening
    total_signal = component1 + component2 + component3
    
    # Add realistic baseline and noise
    baseline = 0.02 + 0.0001 * (field_data - 3350)
    noise = 0.03 * np.random.randn(len(field_data))
    
    intensity_data = total_signal + baseline + noise
    
    # Fake parameters
    params_data = {
        'MWFQ': 9.4,
        'Temperature': 5,
        'Sample': 'Synthetic Er³⁺ multisite'
    }
    
    print(f"Synthetic spectrum created with 3 components")
    print(f"Component centers: 3340, 3355, 3375 G")

# Initial visualization
fig, ax = plt.subplots(figsize=(12, 8))

ax.plot(field_data, intensity_data, 'b-', linewidth=1.5, label='EPR Spectrum')
ax.set_xlabel('Magnetic Field (G)')
ax.set_ylabel('EPR Intensity (a.u.)')
ax.set_title(f'EPR Spectrum for Lineshape Analysis\n{params_data.get("Sample", "Unknown Sample")}')
ax.grid(True, alpha=0.3)
ax.legend()

# Add measurement info
info_text = f"""Measurement Info:
Frequency: {params_data.get('MWFQ', 'Unknown')} GHz
Temperature: {params_data.get('Temperature', 'Unknown')} K
Points: {len(field_data)}
Range: {field_data.max()-field_data.min():.0f} G"""

ax.text(0.02, 0.98, info_text, transform=ax.transAxes,
        verticalalignment='top', 
        bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8),
        fontsize=10)

plt.tight_layout()
plt.show()

print("\n✅ EPR spectrum loaded and visualized!")

## 🎯 Single-Component Lineshape Fitting

Let's start with fitting a single lineshape component to understand the fitting process:

In [ ]:
# Single-component fitting demonstration
print("🎯 Single-Component Lineshape Fitting")
print("=" * 40)

# Apply baseline correction first
try:
    corrected_intensity, fitted_baseline = epyr.baseline.correction.polynomial(
        field_data, intensity_data, params_data,
        order=1,
        exclude_center=True,
        center_fraction=0.4
    )
    print("Baseline correction applied")
except:
    # Simple baseline correction
    baseline_points = np.concatenate([intensity_data[:50], intensity_data[-50:]])
    baseline_level = np.mean(baseline_points)
    corrected_intensity = intensity_data - baseline_level
    fitted_baseline = np.full_like(intensity_data, baseline_level)
    print("Simple baseline correction applied")

# Try different single-component fits
lineshape_types = ['gaussian', 'lorentzian', 'voigtian']
fit_results = {}

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

# Plot original data
axes[0].plot(field_data, intensity_data, 'b-', alpha=0.7, label='Original')
axes[0].plot(field_data, fitted_baseline, 'r--', alpha=0.7, label='Baseline')
axes[0].plot(field_data, corrected_intensity, 'k-', linewidth=2, label='Corrected')
axes[0].set_xlabel('Magnetic Field (G)')
axes[0].set_ylabel('EPR Intensity (a.u.)')
axes[0].set_title('Baseline Correction')
axes[0].grid(True, alpha=0.3)
axes[0].legend()

# Fit each lineshape type
for i, shape_type in enumerate(lineshape_types):
    print(f"\nFitting {shape_type} lineshape...")
    
    try:
        # Use the EPyR Tools fitting function
        if hasattr(epyr.lineshapes, 'fit_epr_signal'):
            fit_result = fit_epr_signal(
                field_data, corrected_intensity,
                lineshape=shape_type,
                n_components=1,
                initial_guess='auto'
            )
            
            fitted_curve = fit_result.fitted_curve
            parameters = fit_result.parameters
            fit_quality = fit_result.r_squared
            
        else:
            # Manual fitting using scipy.optimize
            print(f"  Using manual fitting for {shape_type}...")
            
            # Initial parameter guess
            amplitude_guess = np.max(corrected_intensity)
            center_guess = field_data[np.argmax(corrected_intensity)]
            width_guess = 15  # G
            
            if shape_type == 'gaussian':
                def fit_func(x, amplitude, center, width):
                    return amplitude * gaussian(x, center, width)
                p0 = [amplitude_guess, center_guess, width_guess]
                
            elif shape_type == 'lorentzian':
                def fit_func(x, amplitude, center, width):
                    return amplitude * lorentzian(x, center, width)
                p0 = [amplitude_guess, center_guess, width_guess]
                
            elif shape_type == 'voigtian':
                def fit_func(x, amplitude, center, sigma, gamma):
                    return amplitude * voigtian(x, center, sigma, gamma)
                p0 = [amplitude_guess, center_guess, width_guess/2, width_guess/2]
            
            # Perform fit
            try:
                popt, pcov = curve_fit(fit_func, field_data, corrected_intensity, p0=p0)
                fitted_curve = fit_func(field_data, *popt)
                
                # Calculate R-squared
                ss_res = np.sum((corrected_intensity - fitted_curve) ** 2)
                ss_tot = np.sum((corrected_intensity - np.mean(corrected_intensity)) ** 2)
                fit_quality = 1 - (ss_res / ss_tot)
                
                # Store parameters
                if shape_type == 'voigtian':
                    parameters = {
                        'amplitude': popt[0],
                        'center': popt[1],
                        'sigma': popt[2],
                        'gamma': popt[3]
                    }
                else:
                    parameters = {
                        'amplitude': popt[0],
                        'center': popt[1],
                        'width': popt[2]
                    }
                
            except Exception as fit_error:
                print(f"    Fitting failed: {fit_error}")
                fitted_curve = np.zeros_like(field_data)
                parameters = {}
                fit_quality = 0
        
        # Store results
        fit_results[shape_type] = {
            'fitted_curve': fitted_curve,
            'parameters': parameters,
            'r_squared': fit_quality
        }
        
        # Plot result
        ax = axes[i + 1]
        ax.plot(field_data, corrected_intensity, 'b-', linewidth=1.5, label='Data', alpha=0.7)
        ax.plot(field_data, fitted_curve, 'r-', linewidth=2, label=f'{shape_type.capitalize()} fit')
        ax.plot(field_data, corrected_intensity - fitted_curve, 'g-', alpha=0.7, label='Residual')
        
        ax.set_xlabel('Magnetic Field (G)')
        ax.set_ylabel('EPR Intensity (a.u.)')
        ax.set_title(f'{shape_type.capitalize()} Fit (R² = {fit_quality:.3f})')
        ax.grid(True, alpha=0.3)
        ax.legend()
        
        # Print parameters
        print(f"  R-squared: {fit_quality:.4f}")
        for param, value in parameters.items():
            print(f"  {param}: {value:.3f}")
            
    except Exception as e:
        print(f"  Error fitting {shape_type}: {e}")
        fit_results[shape_type] = {'r_squared': 0}

plt.tight_layout()
plt.show()

# Compare fit qualities
print(f"\n📊 Single-Component Fit Comparison:")
print(f"{'Lineshape':<15} {'R-squared':<12} {'Quality':<15}")
print(f"{'-'*15} {'-'*12} {'-'*15}")

best_fit = None
best_r2 = 0

for shape_type, result in fit_results.items():
    r2 = result.get('r_squared', 0)
    if r2 > 0.8:
        quality = "Excellent"
    elif r2 > 0.6:
        quality = "Good"
    elif r2 > 0.4:
        quality = "Fair"
    else:
        quality = "Poor"
    
    print(f"{shape_type.capitalize():<15} {r2:<12.4f} {quality:<15}")
    
    if r2 > best_r2:
        best_r2 = r2
        best_fit = shape_type

print(f"\n🏆 Best single-component fit: {best_fit.capitalize()} (R² = {best_r2:.4f})")

print("\n✅ Single-component fitting analysis complete!")

## 🔄 Multi-Component Lineshape Fitting

Real EPR spectra often contain multiple overlapping components. Let's demonstrate multi-component fitting:

In [ ]:
# Multi-component fitting
print("🔄 Multi-Component Lineshape Fitting")
print("=" * 40)

# Try fitting multiple Gaussian components
n_components_to_try = [2, 3]
multifit_results = {}

for n_comp in n_components_to_try:
    print(f"\nTrying {n_comp}-component Gaussian fit...")
    
    try:
        # Use EPyR Tools multi-component fitting if available
        if hasattr(epyr.lineshapes, 'fit_multiple_shapes'):
            multifit_result = epyr.lineshapes.fit_multiple_shapes(
                field_data, corrected_intensity,
                lineshapes=['gaussian'] * n_comp,
                initial_centers='auto'
            )
            
            fitted_total = multifit_result.total_fit
            individual_components = multifit_result.components
            fit_quality = multifit_result.r_squared
            parameters = multifit_result.parameters
            
        else:
            # Manual multi-component fitting
            print(f"  Using manual multi-component fitting...")
            
            # Define multi-component function
            if n_comp == 2:
                def multifit_func(x, a1, c1, w1, a2, c2, w2):
                    return (a1 * gaussian(x, c1, w1) + 
                            a2 * gaussian(x, c2, w2))
                
                # Initial guess - find two main peaks
                peak_indices = np.argsort(corrected_intensity)[-2:]
                p0 = [
                    corrected_intensity[peak_indices[0]], field_data[peak_indices[0]], 15,
                    corrected_intensity[peak_indices[1]], field_data[peak_indices[1]], 15
                ]
                
            elif n_comp == 3:
                def multifit_func(x, a1, c1, w1, a2, c2, w2, a3, c3, w3):
                    return (a1 * gaussian(x, c1, w1) + 
                            a2 * gaussian(x, c2, w2) +
                            a3 * gaussian(x, c3, w3))
                
                # Initial guess - spread components across spectrum
                field_range = field_data.max() - field_data.min()
                centers = [field_data.min() + field_range * (i+1)/(n_comp+1) for i in range(n_comp)]
                amps = [np.max(corrected_intensity) * 0.7] * n_comp
                widths = [15] * n_comp
                
                p0 = []
                for i in range(n_comp):
                    p0.extend([amps[i], centers[i], widths[i]])
            
            # Perform fit with bounds
            try:
                # Set reasonable bounds
                if n_comp == 2:
                    bounds = ([0, field_data.min(), 1, 0, field_data.min(), 1],
                             [np.inf, field_data.max(), 50, np.inf, field_data.max(), 50])
                else:  # n_comp == 3
                    lower = [0, field_data.min(), 1] * n_comp
                    upper = [np.inf, field_data.max(), 50] * n_comp
                    bounds = (lower, upper)
                
                popt, pcov = curve_fit(multifit_func, field_data, corrected_intensity, 
                                     p0=p0, bounds=bounds, maxfev=2000)
                
                fitted_total = multifit_func(field_data, *popt)
                
                # Extract individual components
                individual_components = []
                parameters = []
                
                for i in range(n_comp):
                    idx = i * 3
                    amp, center, width = popt[idx], popt[idx+1], popt[idx+2]
                    component = amp * gaussian(field_data, center, width)
                    individual_components.append(component)
                    
                    parameters.append({
                        'amplitude': amp,
                        'center': center,
                        'width': width
                    })
                
                # Calculate R-squared
                ss_res = np.sum((corrected_intensity - fitted_total) ** 2)
                ss_tot = np.sum((corrected_intensity - np.mean(corrected_intensity)) ** 2)
                fit_quality = 1 - (ss_res / ss_tot)
                
            except Exception as fit_error:
                print(f"    Multi-component fitting failed: {fit_error}")
                fitted_total = np.zeros_like(field_data)
                individual_components = []
                parameters = []
                fit_quality = 0
        
        # Store results
        multifit_results[n_comp] = {
            'fitted_total': fitted_total,
            'components': individual_components,
            'parameters': parameters,
            'r_squared': fit_quality
        }
        
        print(f"  {n_comp}-component fit R²: {fit_quality:.4f}")
        
        # Print component parameters
        for i, param_dict in enumerate(parameters):
            print(f"    Component {i+1}:")
            for param, value in param_dict.items():
                unit = "G" if "center" in param or "width" in param else "a.u."
                print(f"      {param}: {value:.2f} {unit}")
                
    except Exception as e:
        print(f"  Error in {n_comp}-component fitting: {e}")
        multifit_results[n_comp] = {'r_squared': 0}

# Plot best multi-component fit
best_multifit = max(multifit_results.items(), key=lambda x: x[1].get('r_squared', 0))
best_n_comp, best_result = best_multifit

if best_result.get('r_squared', 0) > 0:
    print(f"\n📊 Best Multi-Component Fit: {best_n_comp} components")
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
    
    # Top plot: Fit and components
    ax1.plot(field_data, corrected_intensity, 'bo-', linewidth=1.5, markersize=3, 
             alpha=0.7, label='Experimental data')
    ax1.plot(field_data, best_result['fitted_total'], 'r-', linewidth=3, 
             label=f'Total fit (R² = {best_result["r_squared"]:.3f})')
    
    # Plot individual components
    colors = ['green', 'orange', 'purple', 'brown']
    for i, component in enumerate(best_result['components'][:4]):
        ax1.plot(field_data, component, '--', color=colors[i], linewidth=2, 
                 alpha=0.8, label=f'Component {i+1}')
    
    ax1.set_xlabel('Magnetic Field (G)')
    ax1.set_ylabel('EPR Intensity (a.u.)')
    ax1.set_title(f'{best_n_comp}-Component Gaussian Fit')
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    
    # Bottom plot: Residuals
    residuals = corrected_intensity - best_result['fitted_total']
    ax2.plot(field_data, residuals, 'g-', linewidth=1.5, label='Residuals')
    ax2.axhline(y=0, color='red', linestyle='--', alpha=0.7, label='Zero line')
    
    # Calculate residual statistics
    rms_residual = np.sqrt(np.mean(residuals**2))
    max_residual = np.max(np.abs(residuals))
    
    ax2.set_xlabel('Magnetic Field (G)')
    ax2.set_ylabel('Residual (a.u.)')
    ax2.set_title(f'Fit Residuals (RMS: {rms_residual:.3f}, Max: {max_residual:.3f})')
    ax2.grid(True, alpha=0.3)
    ax2.legend()
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n🎯 Multi-Component Analysis Summary:")
    print(f"Best fit: {best_n_comp} components with R² = {best_result['r_squared']:.4f}")
    print(f"RMS residual: {rms_residual:.4f}")
    
    # Component analysis
    print(f"\nComponent Details:")
    total_intensity = 0
    for i, params in enumerate(best_result['parameters']):
        center = params.get('center', 0)
        width = params.get('width', 0)
        amplitude = params.get('amplitude', 0)
        
        # Estimate g-value if microwave frequency is known
        mw_freq = params_data.get('MWFQ', 9.4)  # GHz
        if center > 0 and mw_freq > 0:
            g_value = (mw_freq * 1000) / (center * 0.0000467)  # Rough approximation
        else:
            g_value = 0
        
        # Estimate relative intensity (area under curve)
        relative_intensity = amplitude * width * np.sqrt(np.pi)
        total_intensity += relative_intensity
        
        print(f"  Component {i+1}:")
        print(f"    Center: {center:.1f} G")
        print(f"    Width (FWHM): {width:.1f} G")
        print(f"    Amplitude: {amplitude:.3f}")
        if g_value > 0:
            print(f"    Approximate g-value: {g_value:.3f}")
    
    # Relative intensities
    if total_intensity > 0:
        print(f"\nRelative Intensities:")
        for i, params in enumerate(best_result['parameters']):
            amplitude = params.get('amplitude', 0)
            width = params.get('width', 0)
            component_intensity = amplitude * width * np.sqrt(np.pi)
            percentage = 100 * component_intensity / total_intensity
            print(f"  Component {i+1}: {percentage:.1f}%")

else:
    print("\n❌ Multi-component fitting was not successful")

print("\n✅ Multi-component fitting analysis complete!")

## 📈 Advanced Fitting: Pseudo-Voigt Analysis

For the most realistic EPR lineshape analysis, let's explore Pseudo-Voigt fitting with adjustable Gaussian/Lorentzian mixing:

In [ ]:
# Advanced Pseudo-Voigt fitting
print("📈 Advanced Pseudo-Voigt Lineshape Analysis")
print("=" * 45)

# Define Pseudo-Voigt fitting function with mixing parameter
def fit_pseudo_voigt(x, amplitude, center, width, eta):
    """Pseudo-Voigt with mixing parameter eta (0=pure Gaussian, 1=pure Lorentzian)"""
    return amplitude * pseudo_voigt(x, center, width, eta)

print("\nFitting Pseudo-Voigt lineshape with variable mixing...")

try:
    # Initial parameter guess
    amplitude_guess = np.max(corrected_intensity)
    center_guess = field_data[np.argmax(corrected_intensity)]
    width_guess = 15  # G
    eta_guess = 0.5   # Start with 50-50 mixing
    
    p0 = [amplitude_guess, center_guess, width_guess, eta_guess]
    
    # Set parameter bounds
    bounds = ([0, field_data.min(), 1, 0],  # lower bounds
              [np.inf, field_data.max(), 50, 1])  # upper bounds
    
    # Perform fit
    popt, pcov = curve_fit(fit_pseudo_voigt, field_data, corrected_intensity, 
                          p0=p0, bounds=bounds)
    
    amplitude_fit, center_fit, width_fit, eta_fit = popt
    fitted_pv = fit_pseudo_voigt(field_data, *popt)
    
    # Calculate fit quality
    ss_res = np.sum((corrected_intensity - fitted_pv) ** 2)
    ss_tot = np.sum((corrected_intensity - np.mean(corrected_intensity)) ** 2)
    r_squared_pv = 1 - (ss_res / ss_tot)
    
    # Calculate parameter uncertainties from covariance matrix
    param_errors = np.sqrt(np.diag(pcov))
    
    print(f"\n📊 Pseudo-Voigt Fit Results:")
    print(f"R-squared: {r_squared_pv:.4f}")
    print(f"Amplitude: {amplitude_fit:.3f} ± {param_errors[0]:.3f}")
    print(f"Center: {center_fit:.2f} ± {param_errors[1]:.2f} G")
    print(f"Width (FWHM): {width_fit:.2f} ± {param_errors[2]:.2f} G")
    print(f"Mixing parameter (η): {eta_fit:.3f} ± {param_errors[3]:.3f}")
    
    # Interpret mixing parameter
    if eta_fit < 0.3:
        mixing_desc = "Predominantly Gaussian (inhomogeneous broadening)"
    elif eta_fit > 0.7:
        mixing_desc = "Predominantly Lorentzian (homogeneous broadening)"
    else:
        mixing_desc = "Mixed Gaussian-Lorentzian character"
    
    print(f"\n🔍 Physical Interpretation:")
    print(f"Lineshape character: {mixing_desc}")
    print(f"Gaussian contribution: {(1-eta_fit)*100:.1f}%")
    print(f"Lorentzian contribution: {eta_fit*100:.1f}%")
    
    # Create comprehensive plot
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Main fit plot
    axes[0,0].plot(field_data, corrected_intensity, 'bo-', markersize=3, alpha=0.7, label='Data')
    axes[0,0].plot(field_data, fitted_pv, 'r-', linewidth=3, label=f'Pseudo-Voigt fit')
    
    # Show pure Gaussian and Lorentzian components
    pure_gaussian = amplitude_fit * gaussian(field_data, center_fit, width_fit)
    pure_lorentzian = amplitude_fit * lorentzian(field_data, center_fit, width_fit)
    
    axes[0,0].plot(field_data, pure_gaussian, 'g--', alpha=0.7, 
                   label=f'Pure Gaussian ({(1-eta_fit)*100:.0f}%)')
    axes[0,0].plot(field_data, pure_lorentzian, 'orange', linestyle='--', alpha=0.7,
                   label=f'Pure Lorentzian ({eta_fit*100:.0f}%)')
    
    axes[0,0].set_xlabel('Magnetic Field (G)')
    axes[0,0].set_ylabel('EPR Intensity (a.u.)')
    axes[0,0].set_title(f'Pseudo-Voigt Fit (R² = {r_squared_pv:.3f})')
    axes[0,0].grid(True, alpha=0.3)
    axes[0,0].legend()
    
    # Residuals
    residuals_pv = corrected_intensity - fitted_pv
    axes[0,1].plot(field_data, residuals_pv, 'g-', linewidth=1.5, label='Residuals')
    axes[0,1].axhline(y=0, color='red', linestyle='--', alpha=0.7)
    
    rms_residual_pv = np.sqrt(np.mean(residuals_pv**2))
    axes[0,1].set_xlabel('Magnetic Field (G)')
    axes[0,1].set_ylabel('Residual (a.u.)')
    axes[0,1].set_title(f'Fit Residuals (RMS: {rms_residual_pv:.4f})')
    axes[0,1].grid(True, alpha=0.3)
    
    # Parameter correlation plot
    correlation_matrix = pcov / np.sqrt(np.outer(np.diag(pcov), np.diag(pcov)))
    param_names = ['Amplitude', 'Center', 'Width', 'η (mixing)']
    
    im = axes[1,0].imshow(correlation_matrix, cmap='RdBu_r', aspect='equal', vmin=-1, vmax=1)
    axes[1,0].set_xticks(range(len(param_names)))
    axes[1,0].set_yticks(range(len(param_names)))
    axes[1,0].set_xticklabels(param_names, rotation=45)
    axes[1,0].set_yticklabels(param_names)
    axes[1,0].set_title('Parameter Correlation Matrix')
    
    # Add correlation values
    for i in range(len(param_names)):
        for j in range(len(param_names)):
            axes[1,0].text(j, i, f'{correlation_matrix[i,j]:.2f}', 
                          ha='center', va='center', fontweight='bold')
    
    plt.colorbar(im, ax=axes[1,0])
    
    # Mixing parameter sensitivity analysis
    eta_values = np.linspace(0, 1, 21)
    r2_values = []
    
    for eta_test in eta_values:
        test_curve = amplitude_fit * pseudo_voigt(field_data, center_fit, width_fit, eta_test)
        ss_res_test = np.sum((corrected_intensity - test_curve) ** 2)
        r2_test = 1 - (ss_res_test / ss_tot)
        r2_values.append(r2_test)
    
    axes[1,1].plot(eta_values, r2_values, 'b-', linewidth=2, label='R² vs η')
    axes[1,1].axvline(eta_fit, color='red', linestyle='--', linewidth=2, 
                      label=f'Optimal η = {eta_fit:.3f}')
    axes[1,1].set_xlabel('Mixing Parameter η')
    axes[1,1].set_ylabel('R-squared')
    axes[1,1].set_title('Sensitivity to Mixing Parameter')
    axes[1,1].grid(True, alpha=0.3)
    axes[1,1].legend()
    
    plt.tight_layout()
    plt.show()
    
    # Compare with previous best fits
    print(f"\n🏆 Fit Quality Comparison:")
    print(f"Single Gaussian: R² = {fit_results.get('gaussian', {}).get('r_squared', 0):.4f}")
    print(f"Single Lorentzian: R² = {fit_results.get('lorentzian', {}).get('r_squared', 0):.4f}")
    print(f"Single Voigtian: R² = {fit_results.get('voigtian', {}).get('r_squared', 0):.4f}")
    
    if multifit_results:
        best_multi_r2 = max(result.get('r_squared', 0) for result in multifit_results.values())
        print(f"Multi-component: R² = {best_multi_r2:.4f}")
    
    print(f"Pseudo-Voigt: R² = {r_squared_pv:.4f}")
    
    # Physical insights
    print(f"\n💡 Physical Insights:")
    if eta_fit < 0.3:
        print(f"  - Spectrum dominated by inhomogeneous broadening")
        print(f"  - Possible causes: crystal strain, unresolved hyperfine structure")
    elif eta_fit > 0.7:
        print(f"  - Spectrum dominated by homogeneous broadening")
        print(f"  - Possible causes: spin-lattice relaxation, exchange interactions")
    else:
        print(f"  - Mixed broadening mechanisms")
        print(f"  - Both inhomogeneous and homogeneous effects present")
    
    print(f"  - Linewidth: {width_fit:.1f} G indicates moderate broadening")
    if 'MWFQ' in params_data:
        mw_freq = params_data['MWFQ']
        g_approx = (mw_freq * 1000) / (center_fit * 0.0000467)
        print(f"  - Approximate g-value: {g_approx:.3f}")
    
except Exception as e:
    print(f"❌ Pseudo-Voigt fitting failed: {e}")

print("\n✅ Advanced Pseudo-Voigt analysis complete!")

## 🎯 Lineshape Analysis Best Practices

Let's consolidate the key lessons from EPR lineshape analysis and provide practical guidelines:

In [ ]:
# Comprehensive lineshape analysis summary
print("🎯 EPR LINESHAPE ANALYSIS BEST PRACTICES")
print("=" * 50)

print(f"\n📋 1. DATA PREPARATION CHECKLIST:")
preparation_steps = [
    "Apply proper baseline correction before fitting",
    "Check for phase errors in experimental data",
    "Ensure adequate signal-to-noise ratio (SNR > 10)",
    "Verify field axis calibration and units",
    "Remove obvious artifacts or spurious peaks"
]

for i, step in enumerate(preparation_steps, 1):
    print(f"  {i}. {step}")

print(f"\n🔍 2. LINESHAPE SELECTION GUIDE:")
lineshape_guide = {
    'Gaussian': {
        'Best for': 'Inhomogeneously broadened lines',
        'Physical origin': 'Crystal strain, unresolved hyperfine',
        'Wing behavior': 'Exponential decay',
        'Use when': 'Sharp central peak, fast wing decay'
    },
    'Lorentzian': {
        'Best for': 'Homogeneously broadened lines',
        'Physical origin': 'Lifetime broadening, relaxation',
        'Wing behavior': 'Power law (1/x²)',
        'Use when': 'Extended wings, uniform broadening'
    },
    'Voigtian': {
        'Best for': 'Mixed broadening mechanisms',
        'Physical origin': 'Both inhom. + hom. broadening',
        'Wing behavior': 'Intermediate',
        'Use when': 'Most realistic case, but more parameters'
    },
    'Pseudo-Voigt': {
        'Best for': 'Practical mixed broadening analysis',
        'Physical origin': 'Approximation to Voigtian',
        'Wing behavior': 'Tunable via η parameter',
        'Use when': 'Need mixing ratio, faster computation'
    }
}

for shape, properties in lineshape_guide.items():
    print(f"\n  {shape}:")
    for prop, desc in properties.items():
        print(f"    {prop}: {desc}")

print(f"\n⚙️ 3. FITTING STRATEGY:")
fitting_strategy = [
    "Start with single-component fits to understand basic lineshape",
    "Compare R² values but don't rely solely on fit quality",
    "Use physical constraints (positive amplitudes, reasonable widths)",
    "Check parameter uncertainties and correlations",
    "Validate with residual analysis (random distribution expected)",
    "Consider multi-component fits only when justified",
    "Use Pseudo-Voigt for quantitative broadening mechanism analysis"
]

for i, strategy in enumerate(fitting_strategy, 1):
    print(f"  {i}. {strategy}")

print(f"\n📊 4. QUALITY ASSESSMENT CRITERIA:")
quality_criteria = {
    'Excellent (R² > 0.95)': 'Very good model, low noise, proper lineshape',
    'Good (R² > 0.90)': 'Acceptable fit, minor deviations possible',
    'Fair (R² > 0.80)': 'Reasonable fit, check residuals and parameters',
    'Poor (R² < 0.80)': 'Wrong model, bad data, or insufficient components',
    'Residual patterns': 'Systematic deviations indicate model problems',
    'Parameter uncertainties': 'Should be < 10% of parameter values',
    'Physical reasonableness': 'Parameters must make physical sense'
}

for criterion, description in quality_criteria.items():
    print(f"  {criterion}: {description}")

print(f"\n🔬 5. PHYSICAL PARAMETER EXTRACTION:")
parameter_guide = {
    'g-values': 'g = (h*ν)/(μB*B₀) where ν=MW freq, B₀=resonance field',
    'Linewidths': 'ΔH in Gauss, related to relaxation and interactions',
    'Intensities': 'Proportional to spin concentration (double integral)',
    'Mixing parameter': 'η quantifies homog./inhomog. broadening ratio',
    'Hyperfine splittings': 'From multi-component center positions',
    'Exchange interactions': 'From line broadening and position shifts'
}

for param, description in parameter_guide.items():
    print(f"  {param}: {description}")

print(f"\n⚠️ 6. COMMON PITFALLS:")
pitfalls = [
    "Over-fitting: Using too many components without physical justification",
    "Ignoring baseline: Poor baseline correction leads to parameter errors",
    "Parameter correlation: High correlations indicate over-parameterization",
    "Local minima: Try different initial guesses to find global minimum",
    "Unphysical parameters: Negative amplitudes, unrealistic linewidths",
    "Ignoring uncertainty: Parameter errors tell you about reliability",
    "Model selection bias: Choosing model only based on highest R²"
]

for i, pitfall in enumerate(pitfalls, 1):
    print(f"  {i}. {pitfall}")

print(f"\n🚀 7. ADVANCED TECHNIQUES:")
advanced_techniques = [
    "Global fitting: Fit multiple spectra with shared parameters",
    "Temperature-dependent analysis: Extract activation energies",
    "Angular-dependent fitting: Determine g-tensor components",
    "Derivative lineshapes: First and second derivative analysis",
    "Constrained fitting: Use known physical relationships",
    "Bootstrap analysis: Estimate parameter confidence intervals",
    "Bayesian approaches: Include prior knowledge in fitting"
]

for i, technique in enumerate(advanced_techniques, 1):
    print(f"  {i}. {technique}")

print(f"\n" + "=" * 50)
print(f"✅ LINESHAPE ANALYSIS MASTERY ACHIEVED!")
print(f"You now have the tools for quantitative EPR spectral analysis.")
print(f"=" * 50)

## 🎯 Key Takeaways and Applications

### What You've Mastered:

1. **Lineshape Theory**: Understanding Gaussian, Lorentzian, Voigtian, and Pseudo-Voigt functions
2. **Single-Component Fitting**: Basic spectral analysis and parameter extraction
3. **Multi-Component Analysis**: Resolving overlapping EPR signals
4. **Advanced Fitting**: Pseudo-Voigt analysis for broadening mechanism determination
5. **Quality Assessment**: R², residuals, parameter uncertainties, and physical validation
6. **Physical Interpretation**: Extracting g-values, linewidths, and spin concentrations

### EPyR Tools Workflow Summary:

```python
# Complete lineshape analysis workflow
from epyr import eprload
from epyr.lineshapes import fit_epr_signal, gaussian, pseudo_voigt

# 1. Load and prepare data
field, intensity, params, _ = eprload('spectrum.DSC')
corrected = apply_baseline_correction(field, intensity, params)

# 2. Single-component analysis
result = fit_epr_signal(field, corrected, lineshape='pseudo_voigt')

# 3. Extract parameters
center = result.parameters['center']
width = result.parameters['width']
mixing = result.parameters['eta']

# 4. Physical interpretation
g_value = calculate_g_value(center, params['MWFQ'])
```

### Application Guidelines:

- **Simple Systems**: Start with Gaussian or Lorentzian fits
- **Complex Spectra**: Use Pseudo-Voigt for broadening analysis
- **Multi-Site Systems**: Multi-component Gaussian fitting
- **Quantitative Analysis**: Always include error analysis and physical validation

### Research Applications:

- **Spin Concentration**: From integrated intensities
- **Relaxation Studies**: From homogeneous linewidths
- **Crystal Field Analysis**: From g-value anisotropy
- **Exchange Interactions**: From line broadening effects

### Next Steps:

You're now ready for:
- **Tutorial 05**: CLI tools and automation
- **Advanced Research**: Apply these techniques to your EPR data
- **Publications**: Create publication-quality fitted spectra

## 🚀 Practice Exercises

Perfect your lineshape analysis skills:

### Exercise 1: Systematic Lineshape Comparison
Load multiple EPR spectra and systematically compare different lineshape models. Create a decision matrix for lineshape selection.

### Exercise 2: Multi-Component Resolution
Generate synthetic overlapping Gaussian components with known parameters. Test your ability to recover the original parameters through fitting.

### Exercise 3: Broadening Mechanism Analysis
Use Pseudo-Voigt fits to analyze the temperature dependence of broadening mechanisms in a series of spectra.

### Exercise 4: Parameter Uncertainty Analysis
Implement bootstrap resampling to estimate realistic parameter uncertainties and confidence intervals.

### Exercise 5: Physical Parameter Validation
Calculate g-values, spin concentrations, and relaxation parameters from your fitted results. Validate against known literature values.

---

**🎉 Excellent achievement!** You've mastered quantitative EPR lineshape analysis!

Continue to **[Tutorial 05: Advanced Features](05_Advanced_Features_CLI.ipynb)** to learn automation and advanced workflows.